# __Importing the Data__
Import the root dataset from directory (which came from the NADAC repository) and bring it up to date by appending new data to this file, and export it for use in future modules.

In [2]:
import pandas as pd
from sodapy import Socrata

  Using cached sodapy-2.1.0-py2.py3-none-any.whl (14 kB)


In [19]:
import requests
import json

site = 'https://data.medicaid.gov/resource/a4y5-998d.json'
data = requests.get(site).json()
    
with open('nadac_data.json', 'w') as outfile:
    json.dump(data, outfile)

AttributeError: module 'requests' has no attribute 'get_all'

In [ ]:
# Import most current data from the API
# a4y5-998d --> Weekly data back to February 2013 (the request timed out when I used this endpoint - maybe because the dataset was so big?)
# rt4v-78r4 --> Weekly data back to May 2018

client = Socrata('data.medicaid.gov', app_token='j8YiuHl0sB0jl33nUyw9zQvPL')
client.timeout = 50
results = client.get_all('a4y5-998d', limit = 100000)
client.close()
recent_prices = pd.DataFrame.from_records(results)


In [68]:
recent_prices['effective_date'].max()

'2019-05-15T00:00:00.000'

In [ ]:
recent_prices = recent_prices[recent_prices['effective_date']<'2020-10-21T00:00:00.000']
len(recent_prices)

In [ ]:
#------------------------#
# FOR USE AFTER DOWNLOAD #
#------------------------#

price_root_data = pd.read_csv('NADAC_Source_Data.csv', encoding='utf-8')

# Or download it from the URL (slower - 5 minutes?)
# price_root_data = pd.read_csv('https://data.medicaid.gov/api/views/a4y5-998d/rows.csv?accessType=DOWNLOAD') # The file is almost 700MB, so it can take some time to download
# price_root_data.info()

Now we'll collect new data from the API, concatenate it to our root data source (the file we just imported), and save it back to the original file.

In [4]:
# Reformat the price_root_data columns so the names match the recent_prices
price_root_data.columns = price_root_data.columns.str.replace(' ', '_').str.lower()

# Concatenate current data with price_root_data
all_price_data = pd.concat([price_root_data, recent_prices], ignore_index = True, sort = True)
# Drop duplicate rows
all_price_data.drop_duplicates(inplace = True)

In [5]:
all_price_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7347443 entries, 0 to 7347442
Data columns (total 12 columns):
as_of_date                                   object
classification_for_rate_setting              object
corresponding_generic_drug_effective_date    object
corresponding_generic_drug_nadac_per_unit    object
effective_date                               object
explanation_code                             object
nadac_per_unit                               object
ndc                                          object
ndc_description                              object
otc                                          object
pharmacy_type_indicator                      object
pricing_unit                                 object
dtypes: object(12)
memory usage: 728.7+ MB


In [6]:
# Checking head and tail of the dataframe for a visual check that data types are lining up as expected
all_price_data.head()

,as_of_date,classification_for_rate_setting,corresponding_generic_drug_effective_date,corresponding_generic_drug_nadac_per_unit,effective_date,explanation_code,nadac_per_unit,ndc,ndc_description,otc,pharmacy_type_indicator,pricing_unit
0,09/18/2019,G,NaN,NaN,09/18/2019,1,0.31284,113005452,12-HR DECONGEST 120 MG CAPLET,Y,C/I,EA
1,09/18/2019,G,NaN,NaN,09/18/2019,1,0.31284,36800005452,12-HR DECONGEST 120 MG CAPLET,Y,C/I,EA
2,09/18/2019,G,NaN,NaN,09/18/2019,1,0.31284,36800005460,12-HR DECONGEST 120 MG CAPLET,Y,C/I,EA
3,09/18/2019,G,NaN,NaN,09/18/2019,1,0.31284,24385005452,12HR NASAL DECONGEST ER 120 MG,Y,C/I,EA
4,09/18/2019,G,NaN,NaN,09/18/2019,1,0.31284,70000016401,12HR NASAL DECONGEST ER 120 MG,Y,C/I,EA


In [7]:
all_price_data.tail()

,as_of_date,classification_for_rate_setting,corresponding_generic_drug_effective_date,corresponding_generic_drug_nadac_per_unit,effective_date,explanation_code,nadac_per_unit,ndc,ndc_description,otc,pharmacy_type_indicator,pricing_unit
7347438,2019-05-15T00:00:00.000,B,NaN,NaN,2019-01-01T00:00:00.000,2,8.30666,54123095730,ZUBSOLV 5.7-1.4 MG TABLET SL,N,C/I,EA
7347439,2019-05-15T00:00:00.000,B,NaN,NaN,2019-01-01T00:00:00.000,2,12.40266,54123098630,ZUBSOLV 8.6-2.1 MG TABLET SL,N,C/I,EA
7347440,2019-05-15T00:00:00.000,B,NaN,NaN,2019-01-18T00:00:00.000,"2, 5",47.61993,24208035805,ZYLET EYE DROPS,N,C/I,ML
7347441,2019-05-15T00:00:00.000,B,2019-04-17T00:00:00.000,0.23226,2019-02-19T00:00:00.000,4,40.4315,00002442030,ZYPREXA 20 MG TABLET,N,C/I,EA
7347442,2019-05-15T00:00:00.000,B,NaN,NaN,2019-01-10T00:00:00.000,4,87.62544,57894015012,ZYTIGA 250 MG TABLET,N,C/I,EA


In [3]:
from urllib.request import urlopen # Used to pull the zip file from the site
import os                          # Used to get the current working directory (for data folder)
from zipfile import ZipFile        # Used to unzip the file
from io import BytesIO             # Writes the file to RAM (temporary)
import pandas as pd
import datetime as dt

In [31]:
# Save/Export the new all_price_data dataframe
current_dir = os.getcwd()
data_dir = current_dir + '\data'

#Check if the path doesn't already exist, and create a new 'data' directory if it doesn't
if not os.path.exists(current_dir):
    os.makedirs(data_dir)
print(data_dir)

# Save/Export the new all_price_data dataframe
all_price_data.to_csv('data/NADAC_Current_Data.csv')
print("File saved!")

C:\Users\The Fat Lady\Documents\Python Code\TDI\DrugPricePredictor/data
File saved!


In [34]:
from sodapy import Socrata
import pandas as pd 
from datetime import datetime 

credentials = {}
nadac_parameters = {}
credentials['APP_TOKEN'] = 'j8YiuHl0sB0jl33nUyw9zQvPL'
nadac_parameters['LIMIT'] = 10000000
nadac_parameters['WEBSITE'] = 'data.medicaid.gov'
nadac_parameters['DATA_LOCATION'] = 'a4y5-998d'
nadac_parameters['TIMEOUT'] = 50
nadac_parameters['LAST_DOWNLOADED'] = '2020-08-04T00:00:00.000'
# nadac_parameters['CURRENT_DATE'] = datetime.now().isoformat()


client=Socrata(nadac_parameters['WEBSITE'], credentials['APP_TOKEN'])
client.timeout = int(nadac_parameters['TIMEOUT'])

print('Downloading fresh dataset...')
#Download data
last_downloaded = nadac_parameters['LAST_DOWNLOADED']
current_date = str(datetime.now().date())
print(last_downloaded)
results = client.get(nadac_parameters['DATA_LOCATION'],
#                      where="effective_date between '{}' and '{}'".format(last_downloaded, current_date),
                     content_type='json',
                     limit=int(nadac_parameters['LIMIT']))
df = pd.DataFrame(results)
df.info()

2020-08-04T00:00:00.000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8813197 entries, 0 to 8813196
Data columns (total 12 columns):
 #   Column                                     Dtype 
---  ------                                     ----- 
 0   ndc_description                            object
 1   ndc                                        object
 2   nadac_per_unit                             object
 3   effective_date                             object
 4   pricing_unit                               object
 5   pharmacy_type_indicator                    object
 6   otc                                        object
 7   explanation_code                           object
 8   classification_for_rate_setting            object
 9   as_of_date                                 object
 10  corresponding_generic_drug_nadac_per_unit  object
 11  corresponding_generic_drug_effective_date  object
dtypes: object(12)
memory usage: 806.9+ MB


In [43]:
df['corresponding_generic_drug_nadac_per_unit'].describe()

count      429896
unique      43898
top       0.26108
freq          498
Name: corresponding_generic_drug_nadac_per_unit, dtype: object

In [ ]:
def create_unique_id_index(dataframe, column_1, column_2, column_3):
    df = dataframe.copy()
    df['id'] = df[column_1] + df[column_2] + df[column_3]
    df['id'] = df.apply(lambda row: row['id'].replace('T', ''), axis=1) #remove 'T' (preceeds the time)
    df['id'] = df['id'].str.replace(r'[\W]', '') #Remove any non-alphaneumeric (or underscore) characters
    df['id'] = df['id'].str.replace(r'0{9,}', '') #Remove consecutive zeros with length > 5

    # Print out duplicate rows by index
    duplicate_rows = df[df.duplicated(subset=['id'])]
    print('Count of duplciate rows: ', len(duplicate_rows))
    print('Duplicated rows:', duplicate_rows, sep='\n')
    #Drop duplicates
    df = df[df.drop_duplicates(subset=['id'])]

    #Set id column as index
    df = df.set_index(keys=['id'], drop=True, verify_integrity=True) #checks for duplicates
    
    return dataframe
un_df = create_unique_id_index(df, 'ndc', 'effective_date', 'as_of_date')

In [ ]:
un_df.index.value_counts(dropna=False)

In [51]:
len(temp)

8813197

In [ ]:
temp = temp[temp.drop_duplicates(keep='first')]
len(temp)

In [ ]:
temp.head()

# __Patent Datasets__

Because this dataset comes in three parts, some slight modifications may be necessary prior to merging here.  Code in other notebooks aims to merge this data with the price data.

In [18]:
# Request the file to be downloaded
current_dir = os.getcwd()
data_dir = current_dir + '\data'
zipurl = 'https://www.fda.gov/media/76860/download'

#Check if the path doesn't already exist, and create a new 'data' directory if it doesn't
if not os.path.exists(current_dir):
    os.makedirs(data_dir)

# Download and unzip the patent datasets 
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall(data_dir)

In [19]:
# Read in relevant files (the first 3 belong in a set and will be combined shortly)
drugs = pd.read_csv('data/products.txt', sep='~', engine='python')
patents = pd.read_csv('data/patent.txt', sep='~')
exclusivity = pd.read_csv('data/exclusivity.txt', sep='~')

## __Drugs Dataframe Investigation__

In [15]:
drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37702 entries, 0 to 37701
Data columns (total 14 columns):
Ingredient             37702 non-null object
DF;Route               37702 non-null object
Trade_Name             37702 non-null object
Applicant              37702 non-null object
Strength               37635 non-null object
Appl_Type              37702 non-null object
Appl_No                37702 non-null int64
Product_No             37702 non-null int64
TE_Code                17993 non-null object
Approval_Date          37702 non-null object
RLD                    37702 non-null object
RS                     37702 non-null object
Type                   37702 non-null object
Applicant_Full_Name    37702 non-null object
dtypes: int64(2), object(12)
memory usage: 4.0+ MB


In [14]:
drugs.head()

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,VALEANT PHARMS INTL,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",Yes,Yes,RX,VALEANT PHARMACEUTICALS INTERNATIONAL
1,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,"Nov 26, 2012",No,No,RX,PERRIGO UK FINCO LTD PARTNERSHIP
2,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,"May 24, 2017",No,No,RX,RICONPHARMA LLC
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO,0.12%,A,208204,1,AB,"May 24, 2017",No,No,RX,TARO PHARMACEUTICAL INDUSTRIES LTD
4,CLINDAMYCIN PHOSPHATE,"AEROSOL, FOAM;TOPICAL",CLINDAMYCIN PHOSPHATE,PERRIGO UK FINCO,1%,A,90785,1,AT,"Mar 31, 2010",No,No,RX,PERRIGO UK FINCO LTD PARTNERSHIP


This appears to be an almost entirely complete dataset (with the exception of the 'Strength' and 'TE_Code' columns)!  Let's take a look at the number of unique values in each column (including the NaNs). 

In [16]:
drugs['Appl_No'].value_counts(dropna=False)

19630     53
19631     16
4056      15
21703     15
17512     15
62814     14
204824    14
20533     13
20280     13
76187     12
18268     12
206977    12
21402     12
210831    12
209713    12
11719     12
20678     12
21924     12
20734     12
21137     12
21301     12
21210     12
21292     12
552       12
9218      12
21116     12
20616     12
21342     12
21540     11
76752     11
          ..
84557      1
87294      1
18522      1
20569      1
9000       1
84026      1
88120      1
75830      1
84329      1
71728      1
90143      1
18458      1
88088      1
204823     1
77845      1
72162      1
61453      1
84674      1
86610      1
6441       1
200691     1
204785     1
202736     1
83930      1
88024      1
206314     1
75734      1
78921      1
20411      1
71683      1
Name: Appl_No, Length: 22084, dtype: int64

In [17]:
# Interesting that there are 53 drugs with the same application number (Appl_No).  What are they? They do have different strengths, so we'll leave them alone as they're not true duplicates
drugs[drugs['Appl_No']==19630].head(10)

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
12292,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;200MG/100ML,N,19630,31,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12293,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;450MG/100ML,N,19630,37,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12294,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;37MG/100ML;900MG/100ML,N,19630,43,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12295,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;110MG/100ML,N,19630,1,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12296,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;200MG/100ML,N,19630,7,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12297,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;330MG/100ML,N,19630,13,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12298,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;450MG/100ML,N,19630,19,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12299,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.037% IN DEXTROSE 5% AND S...,B BRAUN,5GM/100ML;37MG/100ML;900MG/100ML,N,19630,25,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12302,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.075% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;75MG/100ML;200MG/100ML,N,19630,32,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC
12303,DEXTROSE; POTASSIUM CHLORIDE; SODIUM CHLORIDE,INJECTABLE;INJECTION,POTASSIUM CHLORIDE 0.075% IN DEXTROSE 10% AND ...,B BRAUN,10GM/100ML;75MG/100ML;450MG/100ML,N,19630,38,NaN,"Feb 17, 1988",No,No,RX,B BRAUN MEDICAL INC


## __Exclusivity Data Investigation__

In [19]:
exclusivity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 5 columns):
Appl_Type           1866 non-null object
Appl_No             1866 non-null int64
Product_No          1866 non-null int64
Exclusivity_Code    1866 non-null object
Exclusivity_Date    1866 non-null object
dtypes: int64(2), object(3)
memory usage: 73.0+ KB


In [18]:
exclusivity.head()

,Appl_Type,Appl_No,Product_No,Exclusivity_Code,Exclusivity_Date
0,N,9190,1,ODE-64,"Apr 4, 2021"
1,N,11366,2,ODE-96,"Aug 7, 2022"
2,N,17858,1,ODE-29,"Aug 13, 2019"
3,N,20287,11,NPP,"May 16, 2022"
4,N,20287,10,NPP,"May 16, 2022"


## __Patents Dataframe Investigation__

In [21]:
patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14717 entries, 0 to 14716
Data columns (total 10 columns):
Appl_Type                  14717 non-null object
Appl_No                    14717 non-null int64
Product_No                 14717 non-null int64
Patent_No                  14717 non-null object
Patent_Expire_Date_Text    14717 non-null object
Drug_Substance_Flag        2529 non-null object
Drug_Product_Flag          8019 non-null object
Patent_Use_Code            7910 non-null object
Delist_Flag                73 non-null object
Submission_Date            10407 non-null object
dtypes: int64(2), object(8)
memory usage: 1.1+ MB


In [20]:
patents.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date
0,N,20571,1,6403569,"Apr 28, 2020",NaN,NaN,U-449,NaN,NaN
1,N,20571,2,6403569,"Apr 28, 2020",NaN,NaN,U-449,NaN,NaN
2,N,20610,1,7625884,"Aug 24, 2026",NaN,NaN,U-141,NaN,NaN
3,N,20629,1,6579981,"Jun 17, 2020",NaN,NaN,U-501,NaN,NaN
4,N,20685,1,6689761,"Feb 10, 2021",NaN,NaN,U-554,NaN,NaN


-------------------------------------------------------

**Note:** Each drug application receives a number ('Appl_No').   An application may have many drugs on it, so each of these drugs on an application receive a product number ('Product_No').  Therefore, I can match up the drugs in the exculusitity and patent data files with those in the drugs data file by first comparing application numbers in each, and the secondarily confirming their product numbers.

-------------------------------------------------------

In [21]:
exclusivity_indexed=exclusivity.set_index(['Appl_No', 'Product_No'])
exclusivity_indexed

Appl_Type Exclusivity_Code Exclusivity_Date
Appl_No Product_No                                            
9190    1                  N           ODE-64      Apr 4, 2021
11366   2                  N           ODE-96      Aug 7, 2022
20287   11                 N              NPP     May 16, 2022
        10                 N              NPP     May 16, 2022
        9                  N              NPP     May 16, 2022
...                      ...              ...              ...
212862  1                  N             GAIN     Aug 14, 2029
211673  1                  N             GAIN     Aug 19, 2029
211672  1                  N             GAIN     Aug 19, 2029
207500  1                  N             GAIN      Mar 6, 2027
207501  1                  N             GAIN      Mar 6, 2027

[2003 rows x 3 columns]

In [27]:
patents.columns = map(str.lower, patents.columns)
exclusivity.columns = map(str.lower, exclusivity.columns)
drugs.columns = map(str.lower, drugs.columns)

### __Merging Patent-related Datasets__

In [29]:
# Merged with 'left' because there were some nan patent data that was unneeded
all_patent_data = pd.merge(drugs, patents, on = ['appl_no', 'product_no'], how = 'left')
# all_patent_data.head()

In [30]:
# Merge in the data from exclusivity.txt
all_patent_data = pd.merge(all_patent_data, exclusivity, on=['appl_no', 'product_no'], how='left')
all_patent_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64470 entries, 0 to 64469
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ingredient               64470 non-null  object
 1   df;route                 64470 non-null  object
 2   trade_name               64470 non-null  object
 3   applicant                64470 non-null  object
 4   strength                 64413 non-null  object
 5   appl_type_x              64470 non-null  object
 6   appl_no                  64470 non-null  int64 
 7   product_no               64470 non-null  int64 
 8   te_code                  20575 non-null  object
 9   approval_date            64470 non-null  object
 10  rld                      64470 non-null  object
 11  rs                       64470 non-null  object
 12  type                     64470 non-null  object
 13  applicant_full_name      64470 non-null  object
 14  appl_type_y              27391 non-nul

In [31]:
all_patent_data.head()

,ingredient,df;route,trade_name,applicant,strength,appl_type_x,appl_no,product_no,te_code,approval_date,...,patent_no,patent_expire_date_text,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,SALIX,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MINOCYCLINE HYDROCHLORIDE,"AEROSOL, FOAM;TOPICAL",AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,NaN,"Oct 18, 2019",...,10086080,"Oct 1, 2030",NaN,NaN,U-2647,NaN,"Nov 15, 2019",NaN,NaN,NaN
2,MINOCYCLINE HYDROCHLORIDE,"AEROSOL, FOAM;TOPICAL",AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,NaN,"Oct 18, 2019",...,10137200,"Oct 1, 2030",NaN,NaN,U-2647,NaN,"Nov 15, 2019",NaN,NaN,NaN
3,MINOCYCLINE HYDROCHLORIDE,"AEROSOL, FOAM;TOPICAL",AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,NaN,"Oct 18, 2019",...,10213512,"Oct 1, 2030",NaN,Y,U-2647,NaN,"Nov 15, 2019",NaN,NaN,NaN
4,MINOCYCLINE HYDROCHLORIDE,"AEROSOL, FOAM;TOPICAL",AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,NaN,"Oct 18, 2019",...,10265404,"Oct 1, 2030",NaN,Y,NaN,NaN,"Nov 15, 2019",NaN,NaN,NaN


In [32]:
# Save/Export the new all_patent_data dataframe
current_dir = os.getcwd()
data_dir = current_dir + '\data'

#Check if the path doesn't already exist, and create a new 'data' directory if it doesn't
if not os.path.exists(current_dir):
    os.makedirs(data_dir)
print(data_dir)

# Save/Export the new patent_data dataframe
all_patent_data.to_csv('data/patent_data.csv')
print("File saved!")

C:\Users\The Fat Lady\Documents\Python Code\TDI\DrugPricePredictor\data
File saved!
